# PyTorch Model Build On Top of VGG 16

---
## Objective
Create a pipeline to feed a neural network image classifier utilizing the VGG 16 backbone.

- Data:
    - [Kaggle Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats/overview)
- Inputs:
    - NumPy arrays
- Outputs:
    - predicted class name
    - probability of predicted class as a percentage

### Imports

In [ ]:
from collections import Counter, namedtuple
import concurrent.futures
from pathlib import Path
from typing import Iterable

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
import seaborn as sns
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torch.utils.data.sampler import WeightedRandomSampler, SubsetRandomSampler
from torchvision import datasets, transforms, models

### Format Notebook

In [ ]:
%matplotlib inline

FONT_SIZE = {
    'label': 14,
    'supertitle': 24,
    'title': 20,
}

---
## Load Data

### Convert Images to NumPy Arrays

This is done only as an exercise to setup a pipeline based on NumPy arrays in lieu of PIL images.

In [ ]:
def image_to_array(im_file: Path):
    """Convert image file to NumPy array."""
    arr_file = im_file.with_suffix('.npy')
    if not arr_file.is_file():
        im = PIL.Image.open(im_file)
        np.save(arr_file, np.array(im))

In [ ]:
data_dir = 'Cat_Dog_data'

im_files = {x.resolve() for x in Path(data_dir).glob('**/*') 
            if x.suffix in ('.jpeg', '.jpg', '.png')}

with concurrent.futures.ProcessPoolExecutor() as executor:
    executor.map(image_to_array, im_files)

### Calculate Mean and Standard Deviation of Dataset

Default ImageNet values over three channels:
$$\mu = (0.485, 0.456, 0.406)$$
$$\sigma = (0.229, 0.224, 0.225)$$

These parameters are appropriate to use for transfer learning when the custom dataset is similar to the ImageNet dataset.

#### Welford's Online Algorithm for Variance

- *Online Algorithm*: An algorithm designed to process each new piece of data as it arrives to produce a final result without knowledge of any future data.
- Calculates the standard deviation in one pass of the data eliminating the need to first cycle through the data to determine the mean.
- Avoids the error found in naive variance calculations when the standard deviation is much smaller than the mean.

[Welford, B.P., 1962. Note on a method for calculating corrected sums of squares and products. Technometrics, 4(3), pp.419-420.](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.302.7503&rep=rep1&type=pdf)

#### Create Datasets

**NOTE**: The images are not resized since they will be yielded from the loader with a batch size of one for the mean and standard caclculation.

##### Datasets From Images Files

In [ ]:
train_im_data = datasets.ImageFolder(f'{data_dir}/train',
                                     transform=transforms.ToTensor())
test_im_data = datasets.ImageFolder(f'{data_dir}/test',
                                    transform=transforms.ToTensor())

##### Datasets From NumPy Arrays

In [ ]:
def npy_loader(path: Path):
    """Helper function to load NumPy files into DataLoader."""
    arr = np.load(path).astype(np.float64)
    np.divide(arr, 255.0, out=arr)
    arr = np.moveaxis(arr, 2, 0)
    return torch.from_numpy(arr).type(torch.FloatTensor)

In [ ]:
train_data = datasets.DatasetFolder(
    root=f'{data_dir}/train',
    loader=npy_loader,
    extensions=('.npy'),
)
test_data = datasets.DatasetFolder(
    root=f'{data_dir}/test', 
    loader=npy_loader,
    extensions=('.npy'),
)

#### Welford's Online Algorithm

In [ ]:
def online_mean_std(loader):
    """Compute the mean and standard deviation in an online fashion."""
    px_cnt = 0
    moment_0 = torch.empty(3)
    moment_1 = torch.empty(3)

    for data in loader:
        data = data[0]
        batch, chanels, height, width = data.shape
        total_pixels = px_cnt + batch * height * width
        channel_sum = torch.sum(data, dim=[0, 2, 3])
        channel_sum_squares = torch.sum(data ** 2, dim=[0, 2, 3])
        moment_0 = (px_cnt * moment_0 + channel_sum) / total_pixels
        moment_1 = (px_cnt * moment_1 + channel_sum_squares) / total_pixels
        px_cnt = total_pixels

    return moment_0, torch.sqrt(moment_1 - moment_0 ** 2)

In [ ]:
Stats = namedtuple('Stats', 'mean, std')

stats = {}
for dataset in (train_data, test_data):
    loader = DataLoader(
        dataset,
        batch_size=1,
        num_workers=1,
        shuffle=False,
    )

    name = dataset.root.split('/')[-1]
    stats[name] = Stats(*online_mean_std(loader))

print(
    f"""
    Channel Statistics
    
    Mean:
    \tTrain: {[f'{x:6f}' for x in stats['train'].mean.tolist()]}
    \tTest:  {[f'{x:6f}' for x in stats['test'].mean.tolist()]}
    
    Standard Deviation
    \tTrain: {[f'{x:6f}' for x in stats['train'].std.tolist()]}
    \tTest:  {[f'{x:6f}' for x in stats['test'].std.tolist()]}
    """
    .replace("'", '')
)

### Transform Dataset 

In [ ]:
random_transforms = [
    transforms.ColorJitter(
        brightness=(0.1, 0.9),
        contrast=0.5,
    ),
    transforms.RandomHorizontalFlip(),
]
shared_transforms = [
    transforms.ToTensor(),
    transforms.Normalize(mean=stats['train'].mean, std=stats['train'].std),
]

train_transforms = transforms.Compose(
    ([transforms.ToPILImage(),
      transforms.RandomResizedCrop(224),
      transforms.RandomApply(random_transforms, p=0.5),
     ]
     + shared_transforms)
) 

test_transforms = transforms.Compose(
    ([transforms.ToPILImage(),
      transforms.Resize(255),
      transforms.CenterCrop(224),
     ]
     + shared_transforms)
)

#### Create Datasets

##### Datasets From Images Files

In [ ]:
train_im_data = datasets.ImageFolder(f'{data_dir}/train',
                                     transform=train_transforms)
test_im_data = datasets.ImageFolder(f'{data_dir}/test',
                                    transform=test_transforms)

##### Datasets From NumPy Arrays

In [ ]:
train_data = datasets.DatasetFolder(
    root=f'{data_dir}/train',
    loader=npy_loader,
    extensions=('.npy'),
    transform=train_transforms,
)
test_data = datasets.DatasetFolder(
    root=f'{data_dir}/test', 
    loader=npy_loader,
    extensions=('.npy'),
    transform=test_transforms,
)

### Create Loaders

In [ ]:
def targets_df(targets: Iterable[int],
               data: 'DatasetFolder') -> pd.DataFrame:
    """
    Convert targets for dataset to data frame.
    
    :param targets: classification targets
    :param data: parent dataset
    """
    df = (pd.DataFrame(targets, columns=['target'])
          .rename_axis('example')
         )
    df['class'] = (df['target']
                   .map({v: k for k, v in data.class_to_idx.items()})
                   .astype('category')
                  )
    return df

#### Calculate Class Weights for Train Data

In [ ]:
batch_size = 32
workers = 2

train_df = targets_df(train_data.targets, train_data)
class_sample_cnt = train_df.groupby('target')['target'].size()
weights = 1 / class_sample_cnt
train_df['weights'] = train_df['target'].map(weights)

#### Split Train Dataset into Stratified Train and Validation Datasets

In [ ]:
train_frac = 0.8

train_mask = (train_df
              .sample(frac=train_frac, weights='weights')
              .index
             )
train_df.loc[train_mask, 'split'] = 'train'
train_df['split'] = train_df['split'].fillna('valid').astype('category')

train_idx, valid_idx = [train_df.query(f"split == '{x}'").index.tolist()
                        for x in ('train', 'valid')]

train_sampler = WeightedRandomSampler(
    weights=torch.tensor(train_df['weights']),
    num_samples=batch_size,
    replacement=False,
)
valid_sampler = SubsetRandomSampler(valid_idx)

trainloader = DataLoader(train_data, batch_size=batch_size,
                         num_workers=workers, sampler=train_sampler)
validloader = DataLoader(train_data, batch_size=batch_size,
                         num_workers=workers, sampler=valid_sampler)
testloader = DataLoader(test_data, batch_size=batch_size,
                        num_workers=workers)

#### Plot Train Validation Class Split with Sample Batch

In [ ]:
plt.figure('Class Balance', figsize=(10, 8),
           facecolor='white', edgecolor='black')
rows, cols = (2, 2)
ax0 = plt.subplot2grid((rows, cols), (0, 0))
ax1 = plt.subplot2grid((rows, cols), (0, 1))
ax2 = plt.subplot2grid((rows, cols), (1, 0))
ax3 = plt.subplot2grid((rows, cols), (1, 1))

sns.countplot(
    x='class',
    hue='split',
    data=train_df,
    palette=sns.color_palette('Paired', n_colors=2),
    ax=ax0
)
ax0.set_title('Train / Validation Class Count', fontsize=FONT_SIZE['title'])

for inputs, labels in trainloader:
    batch_df = targets_df(labels.numpy(), train_data)
    sns.countplot(
        x='class',
        data=batch_df,
        alpha=0.7,
        palette=sns.color_palette('Reds',
                                  n_colors=batch_df['class'].unique().size),
        ax=ax1,
    )
ax1.set_title('Sample Train Batch Class Count', fontsize=FONT_SIZE['title'])
    
sns.countplot(
    x='class',
    data=train_df.query("split == 'train'"),
    color=sns.color_palette('Paired', n_colors=2).as_hex()[0],
    ax=ax2
)
ax2.set_title('Train Class Distribution', fontsize=FONT_SIZE['title'])

sns.countplot(
    x='class',
    data=train_df.query("split == 'valid'"),
    color=sns.color_palette('Paired', n_colors=2).as_hex()[1],
    ax=ax3
)
ax3.set_title('Validation Class Distribution', fontsize=FONT_SIZE['title'])

for ax in (ax0, ax1, ax2, ax3):
    ax.set_xlabel('')
    ax.tick_params(axis='x', which='major', labelsize=FONT_SIZE['label'])
    ax.set_ylabel('Count', fontsize=FONT_SIZE['label'])
    ax.get_yaxis().set_ticks([])
 
for p in ax0.patches:
    height = p.get_height()
    ax0.text(p.get_x() + p.get_width() / 2,
             height + 100,
             f'{height:1.0f}',
             ha="center")

for p in ax1.patches:
    height = p.get_height()
    ax1.text(p.get_x() + p.get_width() / 2,
             height + 0.3,
             f'{height:1.0f} ({height / batch_size * 100:1.0f}%)',
             ha="center")

for ax, length in ((ax2, len(train_idx)), (ax3, len(valid_idx))):    
    ax.set_ylabel('', fontsize=FONT_SIZE['label'])
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x() + p.get_width() / 2,
                height + height / 100,
                f'{height / length * 100:1.0f}%',
                ha="center")

sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.show()

---
## Visualize Data

In [ ]:
def plot_image(image, ax=None, title=None, normalize=True):
    """Plot image from Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    image = image.numpy()
    image = np.moveaxis(image, 0, 2)
    
    if normalize:
        mean = stats['train'].mean.numpy()
        std = stats['train'].std.numpy()
        image = std * image + mean
        image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    for border in ('top', 'right', 'left', 'bottom'):
        ax.spines[border].set_visible(False)
    ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')
    ax.set_title(title)

    return ax

### Examples From Each Dataset

In [ ]:
sets = {
    'train': {'data': train_data, 'loader': trainloader},
    'valid': {'data': train_data, 'loader': validloader},
    'test': {'data': test_data, 'loader': testloader},
}

cols = 5
for row, dataset in enumerate(sets):
    dataiter = iter(sets[dataset]['loader'])
    images, labels = dataiter.next()
    fig, ax = plt.subplots(figsize=(15, 4), ncols=cols)
    for idx in range(cols):
        category = sets[dataset]['data'].classes[labels[idx]]
        plot_image(
            images[idx],
            ax=ax[idx],
            title=category,
        )
        plt.suptitle(f'{dataset.capitalize()} Examples:',
                     fontsize=20)

### Detailed Pixel Values for Test Image

#### WARNING
Execution of this cell is on the order of minutes.

In [ ]:
dataiter = iter(sets[dataset]['loader'])
images, labels = dataiter.next()
rgb_im = np.squeeze(images[0].numpy())
titles = [f'{x} Channel' for x in ('Red', 'Green', 'Blue')]

fig = plt.figure(figsize=(1200, 1200))
for idx in range(3):
    ax = fig.add_subplot(1, 3, idx + 1)
    im = rgb_im[idx]
    ax.imshow(im, cmap='gray')
    ax.set_title(titles[idx])
    width, height = im.shape
    thresh = im.max() / 2.5
    for x in range(width):
        for y in range(height):
            val = round(im[x][y], 2) if im[x][y] != 0 else 0
            ax.annotate(str(val), xy=(y, x),
                        horizontalalignment='center',
                        verticalalignment='center',
                        size=8,
                        color='white' if im[x][y] < thresh else 'black')
plt.savefig('test_image_detail.png')

---
## Network Definition

### VGG 16 Backbone

Load the model and freeze the parameters so backprop will not apply to the backbone.

In [ ]:
model = models.vgg16(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

### Configure for CPU or GPU

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Hardware Execution Mode: {str(device).upper()}')

### Classifier Architecture

Log softmax will be the output of the network to allow easy access to class probabilities during the evaulation step.
This results in the criterion being the negative log likelihood loss `NLLLoss`.

If the `CrossEntropyLoss` were to be used then the outputs would be values from the logits and which would require a transformation to yield probabilities.

In [ ]:
%%capture


class Classifier(nn.Module):
    """New classifier layers for model."""
    def __init__(self):
        super().__init__()
        self.category_cnt = 15
        input_size = 25088
        hidden_1 = 4096
        self.fc1 = nn.Linear(input_size, hidden_1)
        self.output = nn.Linear(hidden_1, self.category_cnt)
        
        self.dropout = nn.Dropout(p=0.2)
    
    def forward(self, x):
        """Define forward pass through layers."""
        x = x.view(x.shape[0], -1)
        
        x = self.dropout(F.relu(self.fc1(x)))
        x = F.log_softmax(self.output(x), dim=1)
        return x


model.classifier = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(
    model.classifier.parameters(),
    lr=0.003,
)
model.to(device)

---
## Train Network

#### Note:
The variable `eval_freq` determines how frequently the *entire* validation set will be evaluated.
Reducing this parameter will result in more evaluations and in turn increase training time.

In [ ]:
epochs = 50
eval_freq = len(train_data) // 2
output_dir = Path()
checkpoint_file = output_dir / 'custom_VGG16.pth'

running_loss = 0
steps = 0
valid_loss_min = np.Inf
for epoch in range(epochs):
    for inputs, labels in trainloader:
        model.train()
        optimizer.zero_grad()
        inputs, labels = [x.to(device) for x in (inputs, labels)]        

        logps = model(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
     
        steps += 1
        if steps % eval_freq == 0:
            
            model.eval()
            valid_loss = 0
            accuracy = 0            
            
            with torch.no_grad():
                for inputs, labels in validloader:
                    inputs, labels = [x.to(device) for x in (inputs, labels)]
        
                    logps = model(inputs)
                    batch_loss = criterion(logps, labels)
                    valid_loss += batch_loss.item()
                    
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += (torch.mean(equals.type(torch.FloatTensor))
                                 .item())
            
            train_loss = running_loss / (batch_size * eval_freq)
            valid_loss = valid_loss / len(valid_loader.sampler)
            accuracy = accuracy / len(valid_loader.sampler)
            print(
                f"""
                Epoch:               {epoch + 1}/{epochs}
                Train Loss:          {train_loss:.3f}
                Validation Loss:     {valid_loss:.3f}
                Validation Accuracy: {accuracy * 100:.2f}%
                
                """)
            
            if valid_loss <= valid_loss_min:
                print(f'Validation loss decreased: '
                      f'{valid_loss_min:.6f} --> {valid_loss:.6f}\n'
                      f'\tSaving model ...')

                torch.save(model.state_dict(), checkpoint_file)
                valid_loss_min = valid_loss
            
            running_loss = 0

---
## Test Model

In [ ]:
test_data.classes

In [ ]:
model.load_state_dict(torch.load(checkpoint_file))

test_loss = 0.0
label_map = {n: c for n, c in enumerate(test_data.classes)}
correct = Counter()
total = Counter()

model.eval()
test_loss = 0

with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = [x.to(device) for x in (inputs, labels)]

        logps = model(inputs)
        batch_loss = criterion(logps, labels)
        test_loss += batch_loss.item()

        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        
        for label in labels:
            category = str(label_map[label.item()])
            correct.update(label)
            total.update(label)
        
test_loss = test_loss / len(test_loader.sampler)

print(
    f"""
    Test Dataset
    Loss:           {test_loss:.6f}
    Total Accuracy: {sum(correct.values / sum(total.values) * 100)}%
    """)

print('Accuracy by Class')
for c in range(test_data.classes):
    print(f'\t{c}: {correct[c] / total[c] * 100:1.0f}% '
          f'({correct[c]}/{total[c]})')